In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  1 16:29:52 2023

@author: Alberto
"""

import rasterio
from affine import Affine
# Open the raster dataset
raster_path = r"E:\rastermunicpial\temoaya.tif"
raster_dataset = rasterio.open(raster_path)

# Open the shapefile and get the geometry
deteccion=r"C:\Users\ASUS\Desktop\DELEGACIONES_EDOMEX\Zona_Temoaya.shp"
deteccion_shp=gpd.read_file(deteccion).to_crs("3857")
geometry=[]
zona=[]
long1=[]
box_l=[]
for i in tqdm.tqdm(range(len(deteccion_shp))):
    shape=mapping(deteccion_shp["geometry"][i])
    coord=[(x[0],x[1]) for x in shape.get("coordinates")[0]]
    left,bottom=np.min(coord,axis=0)
    rigth,top=np.max(coord,axis=0)
    window=raster_dataset.window(left,bottom,rigth,top)
    band = raster_dataset.read(2,window=window)
    transform=raster_dataset.meta.get("transform")[0]
    width=band.shape[1]
    width_min=left
    width_max=rigth
    heigth=band.shape[0]
    heigth_min=bottom
    height_max=top
    blac=np.zeros_like(band)
    box=np.array([(int(np.round(map_d(x[0],width_min,width_max,0,width),0)),int(np.round(map_d(x[1],heigth_min,height_max,heigth,0),0))) for x in coord])
    po=cv2.fillPoly(blac,[box],(1));
    masked_image=blac*np.array(band,dtype="float32")
    out_transform=Affine((rigth-left)/width,0,left,0,-(top-bottom)/heigth,top)
    img = np.array(masked_image.copy(),dtype="float32")**2
    f=35000
    img[img<f]=0
    img[img>=f]=255
    contours,hister=cv2.findContours(np.array(img,dtype="uint8"),2,cv2.CHAIN_APPROX_SIMPLE)
    contours1=[(len(x),x) for x in contours if len(x)>=3]
    long=np.array(contours1)[:,0]
    cnts=np.array(contours1)[:,1]
    for j,c in enumerate(cnts):
        area=cv2.contourArea(c)
        if area>50:
            rect = cv.minAreaRect(c)
            box = cv.boxPoints(rect)
            box = np.int0(box)
            box_l.append(Polygon([(map_d(x[0],0,width,width_min,width_max),map_d(x[1],heigth,0,heigth_min,height_max)) for x in box]))
            # cv.drawContours(img,[box],0,(0,0,255),2)
            long1.append(long[j])
            zona.append(deteccion_shp.loc[i,"zona"])
            geometry.append(Polygon([(map_d(x[0][0],0,width,width_min,width_max),map_d(x[0][1],heigth,0,heigth_min,height_max)) for x in c]))

a=gpd.GeoDataFrame({"puntos":long1,"zona":zona},geometry=geometry,crs="3857")
a["puntos"]=a["puntos"].astype("int")
a["area"]=a.area
a["rectangulo"]=gpd.GeoSeries(box_l,crs=3857).area
a.to_file(r"E:\rastermunicpial\temoaya/industrias.shp")

b=gpd.GeoDataFrame({"puntos":long1,"zona":zona},geometry=box_l,crs="3857")
b["puntos"]=a["puntos"].astype("int")
b["area"]=a.area
b["rectangulo"]=b.area
b.to_file(r"E:\rastermunicpial\temoaya/industrias_c.shp")



